# Dodge_PPO_Attention

In [1]:
import os
from pathlib import Path
import numpy as np
import datetime
import platform
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from mlagents_envs.environment import UnityEnvironment, ActionTuple
from mlagents_envs.side_channel.engine_configuration_channel\
                             import EngineConfigurationChannel
from mlagents_envs.side_channel.environment_parameters_channel\
                             import EnvironmentParametersChannel


## Setting environments

In [2]:
# Global Setting
cur_dir = os.getcwd()
env_dir = os.path.abspath(os.path.join(cur_dir, "..", "Unity6000_Envs"))
test_dir = os.path.abspath(os.path.join(cur_dir, "temp", "pytorch_output"))


### Pytorch Device

In [3]:
# Pytorch Device
if torch.backends.mps.is_available():
    g_device = torch.device("mps")
elif torch.cuda.is_available():
    g_device = torch.device("cuda")
else:
    g_device = torch.device("cpu")

print(g_device)


mps


### Unity Enviroment

In [4]:
# Unity Enviroment
game = "Dodge_Attention"
os_name = platform.system()

if os_name == 'Linux':
    env_name = os.path.join(env_dir, f"{game}_{os_name}.x86_64")
elif os_name == 'Darwin':
    env_name = os.path.join(env_dir, f"{game}_{os_name}.app")

### Seting parameters for PPO

In [5]:
# Seting parameters for PPO-Attention

vel_state_size = 2
ray_chan_size = 40
ray_feat_size = 4
action_size = 5

RAY_OBS = 0
VEL_OBS = 1

# attention parameter
embed_size = 32
num_heads = 4

load_model = False
train_mode = True

discount_factor = 0.99
learning_rate = 3e-4
n_step = 512 # 128
batch_size = 512 #128
n_epoch = 3
_lambda = 0.95
epsilon = 0.2

run_step = 2000000 if train_mode else 0
test_step = 100000

print_interval = 10
save_interval = 100

# Setting parameters for Dodge Environments
env_static_config = {"ballSpeed": 4, "ballRandom": 0.2, "agentSpeed": 3}
env_dynamic_config = {"boardRadius": {"min":6, "max": 8, "seed": 77},
                      "ballNums": {"min": 10, "max": 15, "seed": 77}}

unity_base_port = 1866

In [6]:
# NN model : Save and Load
date_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
save_path = os.path.join(test_dir, f"saved_models/{game}/PPO_Attention/{date_time}")
Path(save_path).mkdir(parents=True, exist_ok=True)
save_model_path = os.path.join(save_path, 'Dodge_PPO_Attention.ckpt')
# print(f"save_path :{save_path}")
# print(f"save_model_path :{save_model_path}")
load_path = "" # Need to update

## Model for Actor-Critic

In [7]:
class AttentionActorCritic(torch.nn.Module):
    def __init__(self, **kwargs):
        super(AttentionActorCritic, self).__init__(**kwargs)
        self.attn_in = torch.nn.Linear(ray_feat_size, embed_size)
        self.attn_layer = torch.nn.TransformerEncoderLayer(
            d_model=embed_size, nhead=num_heads, batch_first=True,
            dim_feedforward=embed_size, dropout=0)
        self.attn_out = torch.nn.Linear(ray_chan_size * embed_size, 128)

        self.e = torch.nn.Linear(vel_state_size, 128)
        self.d1 = torch.nn.Linear(256, 128)
        self.d2 = torch.nn.Linear(128, 128)
        self.pi = torch.nn.Linear(128, action_size)
        self.v = torch.nn.Linear(128, 1)

    def forward(self, state):
        ray, vel = torch.split(state, ray_chan_size * ray_feat_size, dim=1)

        b = ray.shape[0]
        ray = ray.reshape(b * ray_chan_size, ray_feat_size)
        attn_in = self.attn_in(ray).reshape(b, ray_chan_size, embed_size)
        attn_out = self.attn_layer(attn_in)

        ray_embed = F.relu(self.attn_out(attn_out.reshape(b, -1)))
        vel_embed = F.relu(self.e(vel))

        x = torch.cat((vel_embed, ray_embed), dim=1)
        x = F.relu(self.d1(x))
        x = F.relu(self.d2(x))
        return F.softmax(self.pi(x), dim=-1), self.v(x)

## Agent class


In [8]:
# Agent Class
class PPOAgent:
    def __init__(self):
        self.network = AttentionActorCritic().to(g_device)
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr = learning_rate)
        self.memory = list()
        self.writer = SummaryWriter(save_path)

        if load_model == True:
            print(f"... Load Model from {load_path} ...")
            checkpoint = torch.load(load_path, map_location=g_device)
            self.network.load_state_dict(checkpoint["network"])
            self.optimizer.load_state_dict(checkpoint["optimizer"])

    def get_action(self, state, training = True):
        self.network.train(training)
        pi, _ = self.network(torch.FloatTensor(state).to(g_device))
        action = torch.multinomial(pi, num_samples=1).cpu().numpy()
        return action

    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train_model(self):
        self.network.train()

        state      = np.stack([m[0] for m in self.memory], axis=0)
        action     = np.stack([m[1] for m in self.memory], axis=0)
        reward     = np.stack([m[2] for m in self.memory], axis=0)
        next_state = np.stack([m[3] for m in self.memory], axis=0)
        done       = np.stack([m[4] for m in self.memory], axis=0)
        self.memory.clear()

        state, action, reward, next_state, done = map(lambda x: torch.FloatTensor(x).to(g_device),
                                                        [state, action, reward, next_state, done])
        # prob_old, adv, ret
        with torch.no_grad():
            pi_old, value = self.network(state)
            prob_old = pi_old.gather(1, action.long())

            _, next_value = self.network(next_state)
            delta = reward + (1 - done) * discount_factor * next_value - value
            adv = delta.clone()
            adv, done = map(lambda x: x.view(n_step, -1).transpose(0,1).contiguous(), [adv, done])
            for t in reversed(range(n_step-1)):
                adv[:, t] += (1 - done[:, t]) * discount_factor * _lambda * adv[:, t+1]
            adv = adv.transpose(0,1).contiguous().view(-1, 1)

            ret = adv + value

        # training
        actor_losses, critic_losses = [], []
        idxs = np.arange(len(reward))
        for _ in range(n_epoch):
            np.random.shuffle(idxs)
            for offset in range(0, len(reward), batch_size):
                idx = idxs[offset : offset + batch_size]

                _state, _action, _ret, _adv, _prob_old =\
                    map(lambda x: x[idx], [state, action, ret, adv, prob_old])

                pi, value = self.network(_state)
                prob = pi.gather(1, _action.long())

                # loss function for policy network
                ratio = prob / (_prob_old + 1e-7)
                surr1 = ratio * _adv
                surr2 = torch.clamp(ratio, min=1-epsilon, max=1+epsilon) * _adv
                actor_loss = -torch.min(surr1, surr2).mean()

                # loss function for value network
                critic_loss = F.mse_loss(value, _ret).mean()

                total_loss = actor_loss + critic_loss

                self.optimizer.zero_grad()
                total_loss.backward()
                self.optimizer.step()

                actor_losses.append(actor_loss.item())
                critic_losses.append(critic_loss.item())

        return np.mean(actor_losses), np.mean(critic_losses)


    def save_model(self):
        print(f"... Save Model to {save_path}/ckpt ...")
        torch.save({
            "network" : self.network.state_dict(),
            "optimizer" : self.optimizer.state_dict(),
        }, save_model_path)

    def write_summary(self, score, actor_loss, critic_loss, step):
        self.writer.add_scalar("PPO_Random_run/score", score, step)
        self.writer.add_scalar("PPO_Random_model/actor_loss", actor_loss, step)
        self.writer.add_scalar("PPO_Random_model/critic_loss", critic_loss, step)
        # self.writer.add_scalar("PPO_Random_model/total_loss", total_loss, step)

## Train Model

In [9]:
engine_configuration_channel = EngineConfigurationChannel()
environment_parameters_channel = EnvironmentParametersChannel()
env = UnityEnvironment(file_name=env_name,
                       side_channels=[engine_configuration_channel,
                                      environment_parameters_channel],
                                      base_port=unity_base_port)
env.reset()

# Setup Unitu MLAgent
behavior_name = list(env.behavior_specs.keys())[0]
spec = env.behavior_specs[behavior_name]
engine_configuration_channel.set_configuration_parameters(time_scale=12.0)
for key, value in env_static_config.items():
    environment_parameters_channel.set_float_parameter(key, value)
for key, value in env_dynamic_config.items():
    environment_parameters_channel.set_uniform_sampler_parameters(
                            key, value["min"], value["max"], value["seed"])
dec, term = env.get_steps(behavior_name)
num_worker = len(dec)

# PPO agent
agent = PPOAgent()
actor_losses, critic_losses, scores, episode, score = [], [], [], 0, 0
for step in range(run_step + test_step):
    if step == run_step:
        if train_mode:
            agent.save_model()
        print("TEST START")
        train_mode = False
        engine_configuration_channel.set_configuration_parameters(time_scale=1.0)

    # state = dec.obs[0]
    preprocess = lambda ray, vel: np.concatenate((ray.reshape(-1, ray_chan_size * ray_feat_size), vel), axis=1)
    state = preprocess(dec.obs[RAY_OBS], dec.obs[VEL_OBS])
    action = agent.get_action(state, train_mode)
    action_tuple = ActionTuple()
    action_tuple.add_discrete(action)
    env.set_actions(behavior_name, action_tuple)
    env.step()

    # information from enviroment
    dec, term = env.get_steps(behavior_name)
    done = [False] * num_worker
    # next_state = dec.obs[0]
    next_state = preprocess(dec.obs[RAY_OBS], dec.obs[VEL_OBS])
    reward = dec.reward
    # for id in term.agent_id:
    #     _id = list(term.agent_id).index(id)
    #     done[id] = True
    #     next_state[id] = term.obs[0][_id]
    #     reward[id] = term.reward[_id]
    if len(term):
        next_term_state = preprocess(term.obs[RAY_OBS], term.obs[VEL_OBS])
    for id in term.agent_id:
        _id = list(term.agent_id).index(id)
        done[id] = True
        next_state[id] = next_term_state[_id]
        reward[id] = term.reward[_id]
    score += reward[0]

    if train_mode:
        for id in range(num_worker):
            agent.append_sample(state[id], action[id], [reward[id]], next_state[id], [done[id]])
        # training
        if (step+1) % n_step == 0:
            actor_loss, critic_loss = agent.train_model()
            actor_losses.append(actor_loss)
            critic_losses.append(critic_loss)

    if done[0]:
        episode +=1
        scores.append(score)
        score = 0

        # logging
        if episode % print_interval == 0:
            mean_score = np.mean(scores)
            mean_actor_loss = np.mean(actor_losses) if len(actor_losses) > 0 else 0
            mean_critic_loss = np.mean(critic_losses)  if len(critic_losses) > 0 else 0
            agent.write_summary(mean_score, mean_actor_loss, mean_critic_loss, step)
            actor_losses, critic_losses, scores = [], [], []

            print(f"{episode} Episode / Step: {step} / Score: {mean_score:.2f} / " +\
                    f"Actor loss: {mean_actor_loss:.2f} / Critic loss: {mean_critic_loss:.4f}" )

        # save netwrok model
        if train_mode and episode % save_interval == 0:
            agent.save_model()
env.close()



[UnityMemory] Configuration Parameters - Can be set up in boot.config
    "memorysetup-allocator-temp-initial-block-size-main=262144"
    "memorysetup-allocator-temp-initial-block-size-worker=262144"
    "memorysetup-bucket-allocator-granularity=16"
    "memorysetup-bucket-allocator-bucket-count=8"
    "memorysetup-bucket-allocator-block-size=4194304"
    "memorysetup-bucket-allocator-block-count=1"
    "memorysetup-main-allocator-block-size=16777216"
    "memorysetup-thread-allocator-block-size=16777216"
    "memorysetup-gfx-main-allocator-block-size=16777216"
    "memorysetup-gfx-thread-allocator-block-size=16777216"
    "memorysetup-cache-allocator-block-size=4194304"
    "memorysetup-typetree-allocator-block-size=2097152"
    "memorysetup-profiler-bucket-allocator-granularity=16"
    "memorysetup-profiler-bucket-allocator-bucket-count=8"
    "memorysetup-profiler-bucket-allocator-block-size=4194304"
    "memorysetup-profiler-bucket-allocator-block-count=1"
    "memorysetup-profiler

## Test the pretrained model

In [ ]:
load_model = True
train_mode = False

load_path = save_model_path

engine_configuration_channel = EngineConfigurationChannel()
environment_parameters_channel = EnvironmentParametersChannel()
env = UnityEnvironment(file_name=env_name,
                       side_channels=[engine_configuration_channel,
                                      environment_parameters_channel],
                                      base_port=unity_base_port)
env.reset()

# Setup Unitu MLAgent
behavior_name = list(env.behavior_specs.keys())[0]
spec = env.behavior_specs[behavior_name]
engine_configuration_channel.set_configuration_parameters(time_scale=1.0)
for key, value in env_static_config.items():
    environment_parameters_channel.set_float_parameter(key, value)
for key, value in env_dynamic_config.items():
    environment_parameters_channel.set_uniform_sampler_parameters(
                            key, value["min"], value["max"], value["seed"])
dec, term = env.get_steps(behavior_name)
num_worker = len(dec)

# PPO agent
agent = PPOAgent()
# actor_losses, critic_losses, scores, episode, score = [], [], [], 0, 0
for step in range(test_step):
    state = dec.obs[0]
    action = agent.get_action(state, train_mode)
    action_tuple = ActionTuple()
    action_tuple.add_discrete(action)
    env.set_actions(behavior_name, action_tuple)
    env.step()

    # information from enviroment
    dec, term = env.get_steps(behavior_name)
    done = [False] * num_worker
    next_state = dec.obs[0]
    reward = dec.reward
    for id in term.agent_id:
        _id = list(term.agent_id).index(id)
        done[id] = True
        next_state[id] = term.obs[0][_id]
        reward[id] = term.reward[_id]
    score += reward[0]

    if train_mode:
        for id in range(num_worker):
            agent.append_sample(state[id], action[id], [reward[id]], next_state[id], [done[id]])
        # training
        if (step+1) % n_step == 0:
            actor_loss, critic_loss = agent.train_model()
            actor_losses.append(actor_loss)
            critic_losses.append(critic_loss)

    if done[0]:
        episode +=1
        scores.append(score)
        score = 0

        # logging
        if episode % print_interval == 0:
            mean_score = np.mean(scores)
            mean_actor_loss = np.mean(actor_losses) if len(actor_losses) > 0 else 0
            mean_critic_loss = np.mean(critic_losses)  if len(critic_losses) > 0 else 0
            agent.write_summary(mean_score, mean_actor_loss, mean_critic_loss, step)
            actor_losses, critic_losses, scores = [], [], []

            print(f"{episode} Episode / Step: {step} / Score: {mean_score:.2f} / " +\
                    f"Actor loss: {mean_actor_loss:.2f} / Critic loss: {mean_critic_loss:.4f}" )
env.close()



In [11]:
save_model_path


'/Users/hyunjae.k/110_HyunJae_Git/2025_Playgrounds/Unity_Robotics_Playgrounds/Agent_Scripts/temp/pytorch_output/saved_models/Dodge_Attention/PPO_Attention/20250924123650/Dodge_PPO_Attention.ckpt'